# `System.Text.Json.JsonElement` text representation

## the `ToString()` of `JsonElement` is not like the default `ToString()` method

We can see (below) that the `JsonDocument.ToString()` method behaves like the default `object.ToString()` method:

In [1]:
#!csharp

using System.Text.Json;

string json = """
{
    "one": 1,
    "two": 2,
    "three": 3
}
""";

JsonDocument jDoc = JsonDocument.Parse(json);

jDoc.ToString()

System.Text.Json.JsonDocument

However, the root `JsonElement` of `JsonDocument` has a `ToString()` method that behaves differently:

In [2]:
#!csharp

jDoc.RootElement.ToString()

{
    "one": 1,
    "two": 2,
    "three": 3
}

The [official docs explain](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsonelement.tostring?view=net-8.0) that this behavior is _by design_:

> For [JsonValueKind.Null](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsonvaluekind?view=net-8.0#system-text-json-jsonvaluekind-null), [String.Empty](https://docs.microsoft.com/en-us/dotnet/api/system.string.empty?view=net-8.0#system-string-empty) is returned.
>
> For [JsonValueKind.True](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsonvaluekind?view=net-8.0#system-text-json-jsonvaluekind-true), [Boolean.TrueString](https://docs.microsoft.com/en-us/dotnet/api/system.boolean.truestring?view=net-8.0#system-boolean-truestring) is returned.
>
> For [JsonValueKind.False](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsonvaluekind?view=net-8.0#system-text-json-jsonvaluekind-false), [Boolean.FalseString](https://docs.microsoft.com/en-us/dotnet/api/system.boolean.falsestring?view=net-8.0#system-boolean-falsestring) is returned.
>
> For [JsonValueKind.String](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsonvaluekind?view=net-8.0#system-text-json-jsonvaluekind-string), the value of [GetString()](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsonelement.getstring?view=net-8.0#system-text-json-jsonelement-getstring) is returned.
>
> For other types, the value of [GetRawText()](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsonelement.getrawtext?view=net-8.0#system-text-json-jsonelement-getrawtext) is returned.

So, clearly, the output of `RootElement.ToString()` is actually the output of `GetRawText` which is [described](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsonelement.getrawtext?view=net-8.0#system-text-json-jsonelement-getrawtext) with the following interesting sentence:

>Gets a string that represents the original input data backing this value.

When Microsoft says “the original input data,” they are not playing around with words. Look at how the line breaks are preserved in this next call to `RootElement.ToString()`:

In [4]:
#!csharp

string json2 = """
{ "one": 4, "two": 42,
    "three": 43 }
""";

JsonDocument jDoc2 = JsonDocument.Parse(json2);

jDoc2.RootElement.ToString()

{ "one": 4, "two": 42,
    "three": 43 }

Microsoft is deliberately sacrificing control over text formatting for the high performance.

## when you want control over text formatting use `JsonNode.ToJsonString()`

The `JsonNode.ToJsonString()` method [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonnode.tojsonstring?view=net-9.0)] is what you are looking for when you want control over JSON text formatting. So, yes, you have to leave `JsonElement` behind in order to get this control (and lower performance):

In [5]:
#!csharp

using System.Text.Json.Nodes;

JsonNode.Parse(json2).ToJsonString(new JsonSerializerOptions { WriteIndented = true })

{
  "one": 4,
  "two": 42,
  "three": 43
}

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼
